In [1]:
from data_gen.models import Models
from data_gen.models_dict_v2 import model_dict
from data_gen.generate_synthetic_df import generate_synthetic_df
import numpy as np
import matplotlib.pyplot as plt

SEED = np.random.seed(1)
number_of_samples = 5000
train_ratio, valiation_ratio, test_ratio = 0.6,0.2,0.2 #i.e. 0.6*number_of_samples for training, etc.

# 1. Generate Data

In [2]:
def generate_data(sample_number, seed):
    # Create synthetic dataframe
    df = generate_synthetic_df(sample_number, seed)

    # Instantiate an object from the class "Models"
    models = Models(model_dict)

    # Calculate the cost and price
    cost = models.calculate_cost(df)
    pricing = models.calculate_pricing(df)

    # Calculate the profit on the synthetic dataframe
    df["profit"] = models.calculate_profit(cost, pricing)

    return df

df = generate_data(number_of_samples, SEED)

# 2. Clean Data

In [3]:
def standardize_one_column(column):
    mean = np.mean(column)
    standard_deviation = np.std(column)
    return (column - mean)/standard_deviation

def standardize(dataframe):
    #Given a pandas dataframe, we standardize every column.
    number_of_columns = len(dataframe.columns)
    for j in range(number_of_columns):
        dataframe.iloc[:,j] = standardize_one_column(dataframe.iloc[:,j])
    
    return dataframe

In [4]:
def obtain_X_Y(df):
    X = df.loc[:, df.columns != 'profit']
    Y = df["profit"]

    X.loc[X["MARITAL_STATUS"]=="Single", "MARITAL_STATUS"] = 0
    X.loc[X["MARITAL_STATUS"]=="Not_Single", "MARITAL_STATUS"] = 0

    X = X.astype(int)
    Y = Y.astype(int)

    return X,Y

In [5]:
X,Y = obtain_X_Y(df)

X_TRAIN = X.iloc[:int(train_ratio*number_of_samples),:]
Y_TRAIN = Y.iloc[:int(train_ratio*number_of_samples)]
X_VALIDATION = X.iloc[int(train_ratio*number_of_samples):int((train_ratio+valiation_ratio)*number_of_samples),:]
Y_VALIDATION = Y.iloc[int(train_ratio*number_of_samples):int((train_ratio+valiation_ratio)*number_of_samples)]
X_TEST = X.iloc[int((train_ratio+valiation_ratio)*number_of_samples):,:]
Y_TEST = Y.iloc[int((train_ratio+valiation_ratio)*number_of_samples):]

# 3. Train the Decision Tree.

In [6]:
def MSE(y, y_hat):
    return np.sum( np.square(y-y_hat) )
    

In [7]:
from sklearn.tree import DecisionTreeRegressor

min_error = 999999999999999999999
best_depth = None
best_regressor = None

for k in range(1,15):
    regressor = DecisionTreeRegressor(criterion="squared_error", max_depth=k)
    regressor = regressor.fit(X_TRAIN, Y_TRAIN)
    Y_VALIDATION_HAT = regressor.predict(X_VALIDATION)
    error = MSE(Y_VALIDATION, Y_VALIDATION_HAT)
    if error < min_error:
        min_error = error
        best_depth = k
        best_regressor = regressor

print(k)
print(min_error)

14
525128.6317843694


In [8]:
Y_TEST_HAT = best_regressor.predict(X_TEST)
error = MSE(Y_TEST_HAT, Y_TEST)
print(error)

561205.7140404767


# 4. Get the Transition Matrix

By a **state**, we mean a leaf in the best regressor trained above.
Our first goal is to get a list of all the states and get the decision path (i.e. given X, what leaf does X fall into?).

It turns out that sklearn has one id associated to each tree node. Our `state` will thus be a list of integers corresponding to these leaf nodes.

In [9]:
def get_state(regressor):
    #Source: https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
    #This method takes our regressor as input and returns an array of boolean plus an array of integers.
    #The array of booleans will have the length same as the total number of nodes and will indicate if each is a leaf.
    #The array of integers will have the same length as the total number of leaves and will correspond to the location of "True" in the boolean array.
    
    regressor_tree = regressor.tree_

    n_nodes = regressor_tree.node_count
    children_left = regressor_tree.children_left
    children_right = regressor_tree.children_right

    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
    while len(stack) > 0:
        # `pop` ensures each node is only visited once
        node_id, depth = stack.pop()
        node_depth[node_id] = depth

        # If the left and right child of a node is not the same we have a split
        # node
        is_split_node = children_left[node_id] != children_right[node_id]
        # If a split node, append left and right children and depth to `stack`
        # so we can loop through them
        if is_split_node:
            stack.append((children_left[node_id], depth + 1))
            stack.append((children_right[node_id], depth + 1))
        else:
            is_leaves[node_id] = True

    return is_leaves, np.nonzero(is_leaves)[0]

is_leaves, states = get_state(best_regressor)
print(is_leaves)
print(states)


[False False False False False False False False False  True  True False
  True  True False False  True  True  True False False False  True  True
  True False False  True  True False  True  True False False False False
  True  True False  True  True False  True False  True  True False False
 False  True  True False  True  True False False  True  True False  True
  True False False False  True  True False False False  True  True False
  True  True False False  True  True False  True  True False False False
 False  True  True False  True  True False False  True  True False  True
  True False False  True  True False False  True  True  True False False
 False False False False  True  True False  True  True False False  True
  True False  True  True False False False  True  True False  True  True
 False False  True  True False  True  True False False False False  True
  True False  True  True  True False False False  True  True False  True
  True False False  True  True False  True  True Fa

In [10]:
def get_profit_list(regressor):
    #this method returns a list such that, the profit of state i is list[i].
    _, states = get_state(best_regressor)
    value_list = np.array(regressor.tree_.value[:,0,0], dtype = np.float64)
    return value_list[states]

get_profit_list(best_regressor)

array([-9.45000000e+01, -1.23333333e+02, -1.91500000e+02, -1.24000000e+02,
       -1.47000000e+02, -1.58000000e+02, -1.86000000e+02, -9.43800000e+01,
       -1.24333333e+02, -1.55000000e+02, -1.52500000e+02, -1.10000000e+02,
       -1.06200000e+02, -1.43500000e+02, -6.89615385e+01, -5.30909091e+01,
       -7.80000000e+01, -1.05666667e+02, -1.55000000e+02, -6.45000000e+01,
       -1.31000000e+02, -9.40000000e+01, -1.34600000e+02, -1.70333333e+02,
       -1.21000000e+02, -8.74814815e+01, -6.21250000e+01, -7.65000000e+01,
       -1.33333333e+02, -1.63000000e+02, -9.00000000e+01, -1.15000000e+02,
       -1.54500000e+02, -2.22000000e+02, -1.75600000e+02, -2.34750000e+02,
       -2.90000000e+02, -2.11000000e+02, -1.79000000e+02, -1.26454545e+02,
       -2.09000000e+02, -2.02500000e+02, -1.50750000e+02, -8.15000000e+01,
       -1.03000000e+02, -1.52000000e+02, -1.06000000e+02, -2.69000000e+02,
       -2.27000000e+02, -1.62000000e+02, -1.69000000e+02, -1.80000000e+02,
       -3.02500000e+01, -

Now let us print, say, the decision path for `X_TEST`.

In [11]:
def get_state_of_one_year(x, regressor, states):
    x_leaf_id = regressor.apply(x)
    x_states = np.empty(len(x), dtype = np.uint16)
    for k in range(len(x)):
        x_states[k] = np.where(states == x_leaf_id[k])[0][0]
    return x_states

get_state_of_one_year(X_TEST, best_regressor, states)

array([245,  66, 342, 169, 221, 305, 302, 324, 286, 365, 145, 269, 115,
       160,  42, 355, 217, 186, 272, 108,  14, 101, 245, 324, 169, 338,
       386, 119,  90,   0,   7, 376, 305, 229, 128, 126,  14, 108, 117,
       125, 119, 100,  57, 245, 112, 274, 131, 270, 115, 185, 368, 270,
       338, 341, 236, 306, 169, 382, 123, 302, 111,  25, 376, 195,  61,
       265, 253, 157, 151,  15, 192, 160, 237, 221, 311, 115, 417, 146,
       168, 403, 338, 372, 123, 227, 125, 166, 370, 162, 300, 348, 273,
        97, 217,   2,  25, 224, 342, 312, 229, 302, 115, 100, 128, 166,
       245, 156, 224, 277, 295, 166, 186, 236, 417, 245, 125, 221, 270,
       100, 354, 312, 322, 258, 305, 123, 124,  56, 108,  12,  16, 224,
       270, 192, 338, 360, 347, 167, 237,  33, 186, 383,   7, 244, 341,
       272, 112, 186, 124, 399, 112, 413,  27, 169, 297,  25, 364,  57,
       233, 296, 128, 119, 217, 342,  25, 115, 224, 332, 115, 132, 372,
       274, 192, 344,  57, 300, 337, 385, 124, 345, 224, 252, 12

Since the data is random, we will use different seeds to generate $k$ pieces of data of same amound of clients. We assume that such is how one client changes over $k$ years. We shall focus on the mechanism of getting the transition probabilities and will ignore the fact that the client does not age by exactly one year old in the next year.

We remark that, in order for the calculation to work, we must have at least one client for every state. This might not hold. Therefore, we use a large group of clients.

In [12]:
k = 50 #number of years
seeds = np.random.choice(1000, k)
#yearly_X = []
yearly_state = []
sample_number = 5000

for i in range(k):
    new_dataframe = generate_data(sample_number, seeds[i])
    x,_ = obtain_X_Y(new_dataframe)
    #yearly_X.append(x)
    yearly_state.append(get_state_of_one_year(x,best_regressor, states))

print(yearly_state[0])

[250 304 223 ... 266  25  89]


In [13]:
def compute_transition_matrix(yearly_state, number_of_states):
    number_of_years = len(yearly_state)
    number_of_clients = len(yearly_state[0])
    transition_matrix = np.zeros((number_of_states,number_of_states), dtype = np.uint16)

    #For each time t, we count the number of clients starting at state l and end at every state.
    #Once this is done, we make each row a probability vector.

    for t in range(number_of_years-1):
        for n in range(number_of_clients):
            state_this_year = int(yearly_state[t][n])
            state_next_year = int(yearly_state[t+1][n])
            transition_matrix[state_this_year,state_next_year] += 1
    
    transition_matrix = transition_matrix.astype(np.float64)
    for l in range(number_of_states):
        row_sum = np.sum(transition_matrix[l])
        if row_sum == 0:
            transition_matrix[l] = (1/number_of_states)*np.ones(len(transition_matrix[l]))
        else:
            transition_matrix[l] = transition_matrix[l]/row_sum
    
    return transition_matrix

transition_matrix = compute_transition_matrix(yearly_state, len(states))
print(transition_matrix)

[[0.00334448 0.00167224 0.00167224 ... 0.         0.         0.00167224]
 [0.00684932 0.         0.         ... 0.         0.         0.        ]
 [0.00576369 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


# 5. Computation of Customer Lifetime Value(CLV)

Fix a client of Intact.
We are now in a position of defining the concept of customer lifetime value and introduce our algorithm to compute it.

Let $A$ with some $\sigma$-algebra be a measurable space, called the **state space**. For each $t \in \{0,1,2,\cdots\}$, the **client state** is an $A$-valued random variable $S_t$, all defined on one common probability space $(\Omega, F, P)$. For fixed bounded measurable **profit function** $f: A \to \mathbb{R}$ and a **discounting factor** $\gamma = \frac{1}{1.15}$, we define, for every non-negative integer $t_0$ and every $a \in A$:

\begin{equation*}
CLV_{t_0}(a) = \mathbb{E}[\sum_{t=t_0 + 1}^{\tau} \gamma^t f(S_t) \mid S_{t_0} = a]
\end{equation*}

Here $\tau$ is a positive finite stopping time indicating the time which client first quits using Intact. Our goal is to compute $CLV_0(a)$ for every $a \in A$. 

[//]: <1. Enlarge the transition matrix with one row at bottom and one column at right. The bottom right corner of the matrix is $1$. Fill the last row with $0$ and the last column with $0.15$. Normalize the matrix so that it remains a transition matrix. The new state we have added represents the probability of client quiting Intact.>


Let $a \in A$ be given. For every $a' \in A$, we have:

\begin{equation*}
P\{S_1=a' \mid S_0=a\} = P\{S_1=a' \mid S_0=a, \text{client remains}\} P\{ \text{client remains} \mid S_0=a\} 
\end{equation*}

The term $P\{S_1=a' \mid S_0=a, \text{client remains}\}$ on the right hand side is taken care of by the transition matrix computed above. We take $P\{ \text{client remains} \mid S_0=a\} = 0.15$. Generate standard uniform $V$. If $V < P\{ \text{client remains} \mid S_0=a\}$, then the client quits. Otherwise, generate independent standard uniform $U$, which looks at the transition matrix and decide the value of $S_1$. Had simulated the state of $S_t$, repeat in this manner to generate the state of $S_{t+1}$. In doing so, we have generating a sample for the integrand. We then compute the expectation using Kolmogorov's strong law of large numbers.

In [14]:
def generate_sample_paths(number_of_paths, transition_matrix, initial_state, years_limit = 100, client_quit_rate = 0.15):
    sample_paths = []
    for _ in range(number_of_paths):
        sample_paths.append(generate_one_sample_path(transition_matrix, initial_state, years_limit, client_quit_rate))
    return sample_paths

def generate_one_sample_path(transition_matrix, initial_state, years_limit = 100, client_quit_rate = 0.15):
    number_of_states = transition_matrix.shape[0]
    assert initial_state < number_of_states, "State cannot be larger than dimension of matrix."
    sample_path = [initial_state]
    while True:
        v = np.random.uniform(low=0.0, high=1.0, size=1)[0]
        if v < client_quit_rate:
            sample_path.append(-1)
            return sample_path
        last_state = sample_path[-1]
        next_state = np.random.choice([i for i in range(number_of_states)], p=transition_matrix[last_state])
        sample_path.append(next_state)
        if len(sample_path) > years_limit:
            return sample_path

In [15]:
def CLV_one_path(path, profit_list, initial_time = 0, discounting_factor = 1/1.15):
    #Let only one sample path be given. We compute the CLV.
    clv = 0
    for t in range(len(path)):
        if path[t] != -1: #i.e. the client does not quit
            clv += discounting_factor**(initial_time + 1 + t)*profit_list[path[t]]
    return clv

def CLV_estimation(profit_list, initial_state, transition_matrix, initial_time = 0, discounting_factor = 1/1.15, number_of_paths = 10**4):
    sample_paths = generate_sample_paths(number_of_paths, transition_matrix, initial_state, years_limit = 100, client_quit_rate = 0.15)
    clv_samples = np.array([CLV_one_path(path, profit_list, initial_time, discounting_factor) for path in sample_paths])
    return np.mean(clv_samples)


In [16]:
CLV_0 = []
for i in range(len(states)):
    CLV_0.append(CLV_estimation(profit_list = get_profit_list(best_regressor), initial_state= i,
transition_matrix = transition_matrix, initial_time= 0, discounting_factor = 1/1.15, number_of_paths = 10))

print(CLV_0)

[-52.30311094060062, -111.97371769747686, -153.03238966037094, -128.36185621470375, -134.19660295536264, -107.38918019300343, -172.0544505195912, -52.66559071316109, -115.03707743786597, -148.11506356337898, -151.80545580093835, -124.0903848525705, -123.74424250154775, -154.50847284541334, -40.31285467508943, -44.356708778754225, -32.720055709241294, -64.16265844245274, -184.85929231189812, -104.38543767294786, -82.35595806754266, -63.37123980050352, -96.00886112698804, -166.85490448412847, -80.06965855220443, -44.67540289180584, -81.89498427841893, -38.69915403489263, -133.3907083433842, -172.59523825331348, -94.82695271882065, -100.08263336008108, -145.6402733117331, -182.39635147432213, -139.65304950967567, -200.4667188523514, -223.0120791529179, -191.09923799344523, -152.57707421477582, -82.04866443518664, -176.69423265711018, -153.34165774343734, -97.73342725339367, -76.27468452309603, -76.90304461432035, -102.35513616223892, -66.0043539294268, -192.97196380805642, -224.6020579364